<a href="https://colab.research.google.com/github/Justus-coded/Competitions/blob/master/DSN%20AI%20Bootcamp%20Qualification%20Hackathon/DSN%20AI%20Boot_camp%20Qualification1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 
%matplotlib inline
 
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Train.csv')
test  = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/Test.csv')

#train = pd.read_csv('../input/dsn-ai-bootcamp-data/Train.csv')
#test = pd.read_csv('../input/dsn-ai-bootcamp-data/Test.csv')
#sub = pd.read_csv('../input/dsn-ai-bootcamp-data/SampleSubmission.csv')

In [4]:
dict_ = {'charge': 1,
         'lending': 0}
 
target_map = {
    'yes': 1,
    'no': 0}
 
train.default_status.replace(target_map, inplace = True)
 
train['form_field47'].replace(dict_, inplace = True)
test['form_field47'].replace(dict_, inplace = True)

In [7]:
pip install catboost

     |████████████████████████████████| 66.1MB 44kB/s 


In [8]:
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier

In [9]:
features = train.select_dtypes(exclude = object).columns.drop(['default_status'])

In [10]:
# features

In [11]:
for col in features:
    train[col].fillna(-999, inplace = True)
    test[col].fillna(-999, inplace = True)

In [12]:
train['form_field1*formfield2'] = train['form_field1'] * train['form_field2']
train['form_field1/formfield2'] = train['form_field1'] / train['form_field2']

train['form_field1*formfield3'] = train['form_field1'] * train.form_field3
train['form_field1*formfield4'] = train.form_field1 * train.form_field4
train['form_field1*formfield5'] = train.form_field1 * train.form_field5

#new_train['form_field30*formfield41'] = new_train['form_field30'] * new_train['form_field41']
##new_train['form_field31*formfield40'] = new_train['form_field31'] * new_train['form_field40']

train['form_field6+field7'] =  (train.form_field6 + train.form_field8)/2

train['form_field1*formfield6'] = train['form_field1'] * train['form_field6']
train['form_field2*formfield6'] = train['form_field2'] * train['form_field6']

train['form_field1*formfield9'] = train['form_field1'] * train['form_field9']
train['form_field2*formfield9'] = train['form_field2'] * train['form_field9']

train['form_field1*formfield37'] = train['form_field1'] * train['form_field37']
train['form_field2*formfield37'] = train['form_field2'] * train['form_field37']

train['form_field9*formfield6'] = train['form_field9'] * train['form_field6']
train['form_field37*formfield6'] = train['form_field37'] * train['form_field6']

train['form_field9*formfield37'] = train['form_field9'] * train['form_field37']
train['form_field1*formfield8'] = train['form_field1'] * train['form_field8']

In [13]:
test['form_field1*formfield2'] = test['form_field1'] * test['form_field2']
test['form_field1/formfield2'] = test['form_field1'] / test['form_field2']

test['form_field1*formfield3'] = test['form_field1'] * test.form_field3
test['form_field1*formfield4'] = test.form_field1 * test.form_field4
test['form_field1*formfield5'] = test.form_field1 * test.form_field5

test['form_field6+field7'] =  (test.form_field6 + test.form_field8)/2

test['form_field1*formfield6'] = test['form_field1'] * test['form_field6']
test['form_field2*formfield6'] = test['form_field2'] * test['form_field6']

test['form_field1*formfield9'] = test['form_field1'] * test['form_field9']
test['form_field2*formfield9'] = test['form_field2'] * test['form_field9']

test['form_field1*formfield37'] = test['form_field1'] * test['form_field37']
test['form_field2*formfield37'] = test['form_field2'] * test['form_field37']

test['form_field9*formfield6'] = test['form_field9'] * test['form_field6']
test['form_field37*formfield6'] = test['form_field37'] *test['form_field6']

test['form_field9*formfield37'] = test['form_field9'] * test['form_field37']
test['form_field1*formfield8'] = test['form_field1'] * test['form_field8']

In [14]:
X = train[features]
y = train['default_status']

In [15]:
from sklearn.model_selection import StratifiedKFold

In [16]:
def metric(y, pred):
    return roc_auc_score(y, pred, labels=[0, 1])

In [34]:
# Specify number of folds
n_skf = 10
kf = StratifiedKFold(n_skf)
 
 
#seed = 42
 
## Manually tweak the hyperparameter, learning rate and n_estimators especially
params = {'max_depth':8,
    'n_estimators': 5000,
    'learning_rate': 0.008,
    'objective': 'CrossEntropy',
      'eval_metric':'AUC',
    
    'early_stopping_rounds': 200,
    'use_best_model': True,
}

In [35]:
score_list = []
score = 0
test_oofs = []
 
for i, (tr_idx, vr_idx) in enumerate(kf.split(X, y)):
    
    xtrain, ytrain = X.loc[tr_idx, features], y.loc[tr_idx]
    xval, yval = X.loc[vr_idx, features], y.loc[vr_idx]
 
    model = CatBoostClassifier(**params, task_type='GPU', random_seed = i)
    model.fit(xtrain, ytrain, eval_set=[(xval,yval)], verbose=100)
    
    p = model.predict_proba(xval)[:, 1]
    sc = metric(yval, p)
    score_list.append(sc)
    score += sc/n_skf
    
    pred = model.predict_proba(test[features])[:, 1]
    test_oofs.append(pred)
 
    print('Fold {} : {}'.format(i, sc))
 
print()
print()
print('Avg log : ', score)

0:	learn: 0.8053272	test: 0.7915943	best: 0.7915943 (0)	total: 28.5ms	remaining: 2m 22s
100:	learn: 0.8354975	test: 0.8185585	best: 0.8185585 (100)	total: 1.92s	remaining: 1m 33s
200:	learn: 0.8415570	test: 0.8226679	best: 0.8226879 (199)	total: 3.71s	remaining: 1m 28s
300:	learn: 0.8461711	test: 0.8249180	best: 0.8249180 (300)	total: 5.45s	remaining: 1m 25s
400:	learn: 0.8499628	test: 0.8262463	best: 0.8262523 (398)	total: 7.18s	remaining: 1m 22s
500:	learn: 0.8533267	test: 0.8272718	best: 0.8272718 (500)	total: 8.92s	remaining: 1m 20s
600:	learn: 0.8566343	test: 0.8283615	best: 0.8283615 (600)	total: 10.7s	remaining: 1m 18s
700:	learn: 0.8594416	test: 0.8290035	best: 0.8290073 (699)	total: 12.4s	remaining: 1m 15s
800:	learn: 0.8621363	test: 0.8295240	best: 0.8295240 (800)	total: 14.1s	remaining: 1m 14s
900:	learn: 0.8646711	test: 0.8299640	best: 0.8299645 (899)	total: 15.9s	remaining: 1m 12s
1000:	learn: 0.8672019	test: 0.8302915	best: 0.8303413 (995)	total: 17.7s	remaining: 1m 10s
1

In [36]:
f"{n_skf} fold CV, score: {score}" #Increase in local cv score should probably lead to increase in Leaderboard score

'10 fold CV, score: 0.8413637018173287'

10 fold CV, score: 0.8410868999561003, LR-0.008, n_estimator = 5000, seed = 42, depth = 9, 

10 fold CV, score: 0.8413637018173287, LR-0.008, n_estimator = 5000, seed = i, depth = 8,

10 fold CV, score: 0.8416527820408346, LR-0.008, n_estimator = 5000, seed = 42, depth = 8, submit6.csv

'10 fold CV, score: 0.8416283637107365', LR-0.008, n_estimator = 5000, seed = 42, depth = 7, submit5.csv

'10 fold CV, score: 0.8404761359972677', LR-0.008, n_estimator = 5000, seed = 42, depth = 10, submit4.csv

'10 fold CV, score: 0.8416127641119365', LR-0.008, n_estimator = 5000, seed = 2020

'10 fold CV, score: 0.8415039416559706', LR-0.008, n_estimator = 5000, seed = 42

### Since we are using cross validation, We'll have `n` different predictions (where n is the number of fold)

In [21]:
oof_prediction = pd.DataFrame(test_oofs).T

In [22]:
oof_prediction.columns = ['fold_'+ str(i) for i in range(1, n_skf + 1)]

In [23]:
oof_prediction.head()

,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9,fold_10
0,0.308523,0.297050,0.328651,0.318747,0.330469,0.335896,0.296729,0.318233,0.304234,0.319162
1,0.385282,0.338013,0.346464,0.404410,0.443095,0.376553,0.391792,0.332775,0.429403,0.371969
2,0.352425,0.371132,0.406337,0.404204,0.383050,0.407479,0.398560,0.394022,0.349410,0.377015
3,0.737164,0.755433,0.795847,0.731905,0.752634,0.760238,0.777456,0.755901,0.721503,0.757599
4,0.135668,0.146373,0.148061,0.160418,0.155563,0.156377,0.165380,0.156297,0.181091,0.151856


In [25]:
sub = pd.read_csv('/content/drive/My Drive/DSN AI Bootcamp Qualification Hackathon/SampleSubmission.csv')

In [ ]:
### We take the mean of all 5 predictions to generate submission

In [26]:
sub['default_status'] = np.mean(test_oofs, axis = 0)

In [29]:
#from google import files

In [30]:
sub.to_csv('submit6.csv', index = False)

### Suggestions:
- Base your feature selection on the local cv score i.e When you add, remove or engineer feature check `score` (cell 17)
- Increase number of fold to 10, 15. See how score imporve on leaderboard.
- Try differnt value of learning rate. I'll advice anything < 0.01
- Create as many features as possible.
- Find code for Xgboost and lgbm using this method of modelling, or better still tweak this one (mind you, names of parameters may be slightly different.